### Import libraries and data

In [1]:
import pandas as pd
import csv
import re
import numpy as np

In [2]:
vols = pd.read_csv("volumes.csv")
vols = vols.drop("Unnamed: 0", axis=1)

In [3]:
temps = pd.read_csv("temperatures.csv")
temps = temps.drop("Unnamed: 0", axis=1)

In [4]:
#headers = ["Ru", "Rb", "Rh", "Be", "Ba", "Bk", "Br", "H", "P", "Ge", "Gd", "Ga", "Pr", "Pu", "C",\
#"Pd", "Cd", "Ho", "Mg", "Mo", "Mn", "O", "S", "Eu", "Zr", "Er", "Ni",\
#"Na", "Nb", "Nd", "Fe", "B", "F", "Sr", "N", "Si", "Sn", "Sm", "V", "Sc", "Sb", "Se", "Co",\
#"Cl", "Ca", "Ce", "Xe", "Cs", "Cr", "Cu", "La", "Li", "Tm", "Ti", "Te", "Tb", "Tc", "Yb", "Dy",\
#"I", "Y", "Ag", "Al", "As", "In"]

#headers = ["H","Li","Be","B","C","N","O","F","Na","Mg","Al","Si","P","S","Cl",\
#"K","Ca","Sc","Ti","V","Cr","Mn","Fe","Co","Ni","Cu","Zn","Ga",\
#"Ge","As","Se","Br","Rb","Sr","Y","Zr","Nb","Mo",\
#"Tc","Ru","Rh","Pd","Ag","Cd","In","Sn",\
#"Sb","Te","I","Xe","Cs","Ba","La","Ce","Pr",\
#"Nd","Sm","Eu","Gd","Tb","Dy","Ho","Er",\
#"Tm","Yb","Lu","Hf","Ta","W","Re","Os","Ir",\
#"Pt","Au","Hg","Tl","Pb","Bi","Ac","Th","Pa",\
#"U","Np","Am"]

#headers = ["Ru", "Rb", "Rh", "Be", "Ba", "Bi", "Br", "H", "P", "Ge", "Gd", "Ga", "Pr", "Pu", "C",\
#"Pd", "Cd", "Ho", "Mg", "Mo", "Mn", "O", "S", "Eu", "Zr", "Er", "Ni",\
#"Na", "Nb", "Nd", "Fe", "B", "F", "Sr", "N", "Si", "Sn", "Sm", "V", "Sc", "Sb", "Se", "Co",\
#"Cl", "Ca", "Ce", "Xe", "Cs", "Cr", "Cu"]


unitformula = pd.read_csv("scaledformulae.csv")
unitformula = unitformula.drop("Unnamed: 0", axis=1)

### Setting models

In [5]:
from sklearn import linear_model, svm, kernel_ridge
from scipy.optimize import nnls 
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
regVols = linear_model.LinearRegression()
regAlpha = linear_model.LinearRegression()
#regVols = linear_model.ElasticNet(positive = True)

### RMSD function

In [6]:
def getRMSD (y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

#w1 = [(3),(5),(10),(1)]
#w2 = [(2),(4),(6),(8)]
#getRMSD(w1,w2)

### Percentage function

In [7]:
def percCalc (row):
    predictedvolume = row["sum"]
    vols = row["Volume"]
    percentage = predictedvolume / vols
    if (percentage >=0.95) and (percentage <=1.05):
        return 1
    else:
        return 0

### Loop algorithm

In [ ]:
alpha = 0.00
elements = unitformula.columns.values
unitformula["Keep"] = 1
weights = []
alphas =[]
for chunk in zip(unitformula.groupby(np.arange(len(unitformula)) // 10000),vols.groupby(np.arange(len(vols)) // 10000),temps.groupby(np.arange(len(temps)) // 10000)):
    print "Chunk: ", chunk[0][0]
    prevalpha = 1
    RMSD = 10
    chunkformula = chunk[0][1]
    chunkvol = chunk[1][1]
    chunktemp = chunk[2][1]
    keepidx = chunkformula.index.tolist()
    i = 0
    while abs(prevalpha-alpha) > 0.00001 or (RMSD > 1) :
        print "Iteration:", i
        regVols.fit(chunkformula[chunkformula.Keep == 1], chunkvol.loc[keepidx].div(chunktemp.loc[keepidx].Temperature*alpha+1, axis="index"))
        if i > 0:
            prevw = w#[0]
            #print "old w: ", prevw
        w = regVols.coef_[0]
        #w, rnorm = nnls(np.matrix(chunkformula[chunkformula.Keep == 1]),np.squeeze(np.array(chunkvol.loc[keepidx].div(chunktemp.loc[keepidx].Temperature*alpha+1, axis="index"))))
        if i > 0:
            RMSD = getRMSD(prevw,w)#[0]
            print "New w", zip(elements, w)
            print "RMSD: ", RMSD
        x2=pd.DataFrame()
        for line in zip(elements, w): #[0]
            x2[line[0]] = chunkformula[line[0]].apply(lambda x: x*line[1])
        x2["sum"] = x2.sum(axis=1)
        #if i>0:
            #print x2["sum"]
        vol2 = chunkvol.loc[keepidx].div(x2["sum"].loc[keepidx], axis="index") - 1
        regAlpha.fit(chunktemp.loc[keepidx], vol2)
        if i > 0:
            prevalpha = alpha
            print "Previous Alpha: ", prevalpha
        alpha = regAlpha.coef_[0][0]
        #alpha, rnorm2 = nnls(np.matrix(chunktemp.loc[keepidx]),np.squeeze(np.array(vol2)))
        print "Alpha: ", alpha
        print "deltaAlpha: ", abs(prevalpha-alpha)
        #print "dAlpha: ", abs(prevalpha - alpha)
        chunkformula["Keep"] = pd.concat([x2,vols], axis=1).apply(percCalc, axis=1)
        keepidx = chunkformula.Keep[chunkformula.Keep == 1].index.tolist()
        #print "Element average volume (cubic angstroms)", zip(elements,w[0])
        i +=1
        if (len(keepidx) <= 1) or (i > 200):
            print "Failure"
            break
    weights.append(w)#[0])
    alphas.append(alpha)

### Misc

In [11]:
hofmw = np.array([37.3, 42, 31.2, 36, 13.24, 66, 32.7, 5.08, 29.5, 41.6, 56, 37.8, 57,\
13.87, 35, 51, 42, 36, 36, 38, 31.9, 11.39, 25.2, 39, 53, 27, 54, 26,\
26, 37, 50, 30.4, 13.24, 11.17, 47, 11.8, 37.3, 52.8, 50, 24, 42, 48, 30.3,\
29.4, 25.8, 45, 54, 45, 46, 28.1, 26.9, 58, 22.6, 49,\
27.3, 46.7, 45, 38, 59, 50, 46.2, 44, 35, 39.6, 36.4, 55])

In [1]:
vol2.hist(bins=30)
plt.show()

A graph of the volumes divided by the predicted volumes is made; this was just to test out making simple graphs and to view the distribution of volumes

In [ ]:
for line in zip(elements, hofmw):
            x2[line[0]] = chunkformula[line[0]].apply(lambda x: x*line[1])
x2["sumhoff"] = x2.sum(axis=1)
print x2["sum"]
print x2["sumhoff"]

### Misc - Comparison methods

All of the imported functions below are different methods of vector comparisons; these would tell us how similar two sets of values were, in this case it can be the predicted volumes and actual volumes

In [ ]:
from scipy import spatial
from scipy.spatial import distance
cossimi = 1 - spatial.distance.cosine(x2["sumhoff"], chunk[1][1])
print cossimi

This is the cosine similarity; it calculates the cosine of the angle between two n-dimensional vectors; what is expected is an angle of 0 which corresponds to a cosine sim value of 1. A value of 0 means values at 90 degrees to each other; and value of -1 means they are opposed.

The value is subtracted from 1 because the function from scipy is (1 - cosine sim), therefore to revert it.

In [ ]:
eucsimi = spatial.distance.euclidean(x2["sumhoff"], chunk[1][1])
print eucsimi

The eucidean distance is basically the pythagoras distance, so the root of the sum of the differences squared is taken.
$$\sqrt{\sum(A_{i} -B_{i})^{2}}$$

This equation isn't entirely correct, the sum requires a i=1 and n limit, but I couldn't get it to work. This applies for all the equatiosn

In [ ]:
mansimi = distance.minkowski(x2["sumhoff"], chunk[1][1], 1) # manhattan distance 
chebsimi = distance.minkowski(x2["sumhoff"], chunk[1][1], 3) # chebyshev distance
print mansimi, chebsimi

Manhattan distance = |x1 – x2| + |y1 – y2|.
Chebyshev is as euclidean but the square and root power is set to an infinite

In [ ]:
bray = 1 - distance.braycurtis(x2["sumhoff"], chunk[1][1])
print bray
#bray curtis, similar to canberra, however here the weighting isn't absolute and also each component (numerator & denominator)
#is summed before division (compared to canberra where it is summed after division.)

The bray curtis similarity is where the sum of the differences is divided by the sum of each set of values.
A rough equation is shown below
$$1 - \frac{\sum\mid y_{i}-y_{j} \mid}{\sum(y_{i}+y_{j})}$$

This is bound between 0 & 1; a higher value indicates more similarity therefore better

In [ ]:
canb = distance.canberra(x2["sumhoff"], chunk[1][1])
print canb
#canberra, weighted form of manhattan distance (where the abs difference is divided by sum of absolutes of each set)

The canberra similiarty is very similar to the bray curtis similiarty; however there is no max boundary; the minimum value is 0. Here a lower value indicates higher similiarty therefore is better.
$$\sum\frac{\mid y_{i}-y_{j} \mid}{\mid y_{i} \mid + \mid y_{j} \mid}$$

In [ ]:
corr = distance.correlation(x2["sumhoff"], chunk[1][1])
print corr

The distance correlation basically shows how statistically independent two sets are from eachother, where 0 = totally independent and 1 is totally dependent.

In [ ]:
print "The mean value of both our average volume: ", np.mean(w), "and hofmann average volume: ", np.mean(hofmw)